<a href="https://colab.research.google.com/github/Kremlin-dev/Driver_Drowsiness-Detection-FinalYearProject/blob/main/drowsy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dropout, Conv2D, Flatten, Dense, MaxPooling2D, BatchNormalization
import random, shutil

In [2]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True, batch_size=1, target_size=(24, 24), class_mode='categorical'):
    return gen.flow_from_directory(dir, batch_size=batch_size, shuffle=shuffle, color_mode='grayscale', class_mode=class_mode, target_size=target_size)

In [3]:
BS = 32
TS = (24, 24)
train_batch = generator('/content/drive/MyDrive/ROI/dataset/train', shuffle=True, batch_size=BS, target_size=TS)
valid_batch = generator('/content/drive/MyDrive/ROI/dataset/test', shuffle=True, batch_size=BS, target_size=TS)
SPE = len(train_batch.classes) // BS
VS = len(valid_batch.classes) // BS
print(SPE, VS)

Found 1234 images belonging to 2 classes.
Found 218 images belonging to 2 classes.
38 6


In [4]:
model = Sequential([
    Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(24, 24, 1)),
    MaxPooling2D(pool_size=(1, 1)),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1, 1)),
    Dropout(0.25),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.fit(train_batch, validation_data=valid_batch, epochs=15, steps_per_epoch=SPE, validation_steps=VS)

Epoch 1/15
38/38 [==============================] - 14s 293ms/step - loss: 0.4734 - accuracy: 0.7679 - val_loss: 0.2674 - val_accuracy: 0.8958
Epoch 2/15
38/38 [==============================] - 10s 254ms/step - loss: 0.2877 - accuracy: 0.9085 - val_loss: 0.1575 - val_accuracy: 0.9323
Epoch 3/15
38/38 [==============================] - 9s 226ms/step - loss: 0.1904 - accuracy: 0.9376 - val_loss: 0.1472 - val_accuracy: 0.9375
Epoch 4/15
38/38 [==============================] - 10s 253ms/step - loss: 0.1445 - accuracy: 0.9559 - val_loss: 0.1431 - val_accuracy: 0.9375
Epoch 5/15
38/38 [==============================] - 11s 298ms/step - loss: 0.1294 - accuracy: 0.9584 - val_loss: 0.1272 - val_accuracy: 0.9323
Epoch 6/15
38/38 [==============================] - 9s 248ms/step - loss: 0.1016 - accuracy: 0.9667 - val_loss: 0.1147 - val_accuracy: 0.9531
Epoch 7/15
38/38 [==============================] - 9s 239ms/step - loss: 0.0841 - accuracy: 0.9667 - val_loss: 0.2975 - val_accuracy: 0.9062
Ep

In [7]:
tf.saved_model.save(model, '/content/drive/MyDrive/ROI/saved_model')

In [8]:
model.save('/content/drive/MyDrive/ROI/drowsinessv3.h5', overwrite=True)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
def load_representative_data():
    # Load your dataset here
    # Example: loading images from a directory
    import os
    from tensorflow.keras.preprocessing import image

    data_dir = '/content/drive/MyDrive/ROI/dataset/test'
    image_paths = [os.path.join(data_dir, fname) for fname in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, fname))] # Filter to include only files

    for img_path in image_paths:
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array.astype(np.float32) / 255.0  # Normalize if necessary
        yield [img_array]

def representative_dataset():
    for data in load_representative_data():
        yield data

In [17]:
# saved_model_dir = '/content/saved_model'

model = load_model('/content/drive/MyDrive/ROI/drowsinessv3.h5')

In [18]:
# converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Set the optimization strategy
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Set the representative dataset
converter.representative_dataset = representative_dataset

# Ensure full integer quantization for all ops
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8

# Convert the model
tflite_quant_model = converter.convert()

# Save the quantized model
with open('/content/drive/MyDrive/ROI/model_quantized.tflite', 'wb') as f:
    f.write(tflite_quant_model)

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [12]:
model.save('drowsinessv3.h5', overwrite=True)

In [13]:
model.save('drowsinessv3.keras')

In [14]:
tf.saved_model.save(model, '/content/drive/MyDrive/ROI/saved_model')